In [1]:
# ssh -f django@bcpp3.bhp.org.bw -L5003:localhost:3306 -N
# ssh -f django@bhp066.bhp.org.bw -L5002:localhost:3306 -N

In [13]:
# List of missing subject identifier
missing_arvs = []
file_name = '/Users/ckgathi/Downloads/BHP_FILE/check19.csv'
f = open(file_name, 'r')
lines = f.readlines()
lines.pop(0)
for line in lines:
    line = line.strip()
    missing_arvs.append(line)

missing_arvs = list(set(missing_arvs))
print(len(missing_arvs))

164


In [14]:
# List of id for missing avrs data for HivCareAdherence
from bcpp_subject.models import HivCareAdherence
from django_pandas.io import read_frame

hiv_care_adherence = HivCareAdherence.objects.filter(subject_visit__subject_identifier__in=missing_arvs)
print(hiv_care_adherence.count())
df = read_frame(hiv_care_adherence)
hiv_care_adherence_ids = df['id'].tolist()
ids = []
for id in hiv_care_adherence_ids:
    ids.append(id.hex)
ids = list(set(ids))

313


In [4]:
import MySQLdb
import pandas as pd
db_name = 'bcpp051'
connection = MySQLdb.connect(host='127.0.0.1', port=5003,user='root', passwd='cc3721b', db=f'{db_name}')
data_arv = []
data_prev_arvs = []

count_found_arv = 0
count_found_prev_arvs = 0
for idd in ids:
    sql_arvs = f"SELECT CONCAT(bcpp_subject_arv.short_name, bcpp_subject_hivcareadherence_arvs.hivcareadherence_id) as avr_id FROM bcpp_subject_arv INNER JOIN bcpp_subject_hivcareadherence_arvs ON bcpp_subject_arv.id = bcpp_subject_hivcareadherence_arvs.arv_id WHERE bcpp_subject_hivcareadherence_arvs.hivcareadherence_id = \'{idd}\';"
    sql_prev_arvs = f"SELECT CONCAT(bcpp_subject_arv.short_name, bcpp_subject_hivcareadherence_prev_arvs.hivcareadherence_id) as prev_arvs_id FROM bcpp_subject_arv INNER JOIN bcpp_subject_hivcareadherence_prev_arvs ON bcpp_subject_arv.id = bcpp_subject_hivcareadherence_prev_arvs.arv_id WHERE bcpp_subject_hivcareadherence_prev_arvs.hivcareadherence_id = \'{idd}\';"
    df_arvs = pd.read_sql(sql_arvs, con=connection)
    df_prev_arvs = pd.read_sql(sql_prev_arvs, con=connection)
    if not df_arvs.empty:
        arvs_ids_list = df_arvs['avr_id'].tolist()
        if len(arvs_ids_list) > 1:
            count_found_arv += len(arvs_ids_list)
        else:
            count_found_arv += 1
        data_arv.append(arvs_ids_list)
    if not df_prev_arvs.empty:
        prev_arvs_ids_list = df_prev_arvs['prev_arvs_id'].tolist()
        if len(prev_arvs_ids_list) > 1:
            count_found_prev_arvs += len(prev_arvs_ids_list)
        else:
            count_found_prev_arvs += 1
        data_prev_arvs.append(prev_arvs_ids_list)
print('------------------------------------------------')
print(f"Found items avr: {count_found_arv}")
print(data_arv)
print('------------------------------------------------')
print(f"Found items prev_arvs: {count_found_prev_arvs}")
print(data_prev_arvs)
print('------------------------------------------------')

------------------------------------------------
Found items avr: 17
[['Lopinavir/ritonavirb378bbd68cc143fcac81c2d332f92ecb'], ['Lopinavir/ritonavir7fa639271ec945f0b6985b2d65436ed0'], ['Lopinavir/ritonavir12be8cb2914a4d4abdc1b242a48c2745'], ['Tenofovir/emtricitabine1b1b0504ad284482bfaad86d07aff236', 'Nevirapine1b1b0504ad284482bfaad86d07aff236'], ['Lopinavir/ritonavir158bfd8990b54f0b9289c072a74ab53e'], ['Lopinavir/ritonavirac7cc51b37d14fe1a66174e3f2779fe0'], ['Lopinavir/ritonavirf657ef76c15545d6a9809f111b46d01c'], ['Lopinavir/ritonavir513401b4c9bd4b00853c693fa43021c6'], ['Efavirenz27681f8abe364708bb2d304b3aed4b18', 'Zidovudine/lamivudine27681f8abe364708bb2d304b3aed4b18', 'Nevirapine27681f8abe364708bb2d304b3aed4b18'], ['Lopinavir/ritonavir006e8dcaea8e472f95285025421df923'], ['Lopinavir/ritonavire345543f343e4134837815dfdfa63eb6'], ['OTHERe6592cdd36d445c6bf132a7b10481d19'], ['Lopinavir/ritonavirbad2f0b476124ceb91ae25d055643473'], ['Lopinavir/ritonavir4e4646ba71b24cf49aafc702de72a781']]
---

In [5]:
# Update central server
from bcpp_subject.models import Arv
count_moved_avr = 0
ccc_avr = []
items_avr = 0
for d in data_arv:
    for hca_id in d:
        short_name = hca_id[:-32]
        obj_uuid = hca_id[-32:]
        try:
            hca = HivCareAdherence.objects.get(id=obj_uuid)
        except HivCareAdherence.DoesNotExist:
            print(f'this is a problem it needs to be moved to the server {hca_id}')
        else:
            arv = Arv.objects.get(short_name=short_name)
            if not arv in hca.arvs.all():
                hca.arvs.add(arv)
                hca.save_base(raw=True)
                count_moved_avr += 1
                ccc_avr.append(obj_uuid)
                print(short_name, obj_uuid, hca.subject_visit.subject_identifier, hca.subject_visit.household_member.household_structure.household.plot.map_area)
            else:
                print("PRESENT", short_name, obj_uuid, hca.subject_visit.subject_identifier, hca.subject_visit.household_member.household_structure.household.plot.map_area)
        items_avr += 1

PRESENT Lopinavir/ritonavir b378bbd68cc143fcac81c2d332f92ecb 066-16300014-2 lentsweletau
PRESENT Lopinavir/ritonavir 7fa639271ec945f0b6985b2d65436ed0 066-236800045-8 maunatlala
PRESENT Lopinavir/ritonavir 12be8cb2914a4d4abdc1b242a48c2745 066-236800034-2 maunatlala
PRESENT Tenofovir/emtricitabine 1b1b0504ad284482bfaad86d07aff236 066-22270002-5 sefophe
PRESENT Nevirapine 1b1b0504ad284482bfaad86d07aff236 066-22270002-5 sefophe
PRESENT Lopinavir/ritonavir 158bfd8990b54f0b9289c072a74ab53e 066-23910156-3 maunatlala
PRESENT Lopinavir/ritonavir ac7cc51b37d14fe1a66174e3f2779fe0 066-163000027-0 lentsweletau
PRESENT Lopinavir/ritonavir f657ef76c15545d6a9809f111b46d01c 066-205000031-1 mmathethe
PRESENT Lopinavir/ritonavir 513401b4c9bd4b00853c693fa43021c6 066-205100075-7 mmathethe
PRESENT Efavirenz 27681f8abe364708bb2d304b3aed4b18 066-22150027-3 sefophe
PRESENT Zidovudine/lamivudine 27681f8abe364708bb2d304b3aed4b18 066-22150027-3 sefophe
PRESENT Nevirapine 27681f8abe364708bb2d304b3aed4b18 066-22150

In [6]:
# Update central server
from bcpp_subject.models import Arv
count_moved_prev_avr = 0
ccc_prev_avr = []
items_avr = 0
for d in data_prev_arvs:
    for hca_id in d:
        short_name = hca_id[:-32]
        obj_uuid = hca_id[-32:]
        try:
            hca = HivCareAdherence.objects.get(id=obj_uuid)
        except HivCareAdherence.DoesNotExist:
            print(f'this is a problem it needs to be moved to the server {hca_id}')
        else:
            arv = Arv.objects.get(short_name=short_name)
            if not arv in hca.prev_arvs.all():
                hca.prev_arvs.add(arv)
                hca.save_base(raw=True)
                count_moved_prev_avr += 1
                ccc_prev_avr.append(obj_uuid)
                print(short_name, obj_uuid, hca.subject_visit.subject_identifier, hca.subject_visit.household_member.household_structure.household.plot.map_area)
            else:
                print("PRESENT", short_name, obj_uuid, hca.subject_visit.subject_identifier, hca.subject_visit.household_member.household_structure.household.plot.map_area)
        items_avr += 1
        

PRESENT Zidovudine/lamivudine b378bbd68cc143fcac81c2d332f92ecb 066-16300014-2 lentsweletau
PRESENT Zidovudine/lamivudine bad2f0b476124ceb91ae25d055643473 066-205100070-8 mmathethe


In [7]:
# Check if the data has been moved
mis_arv = 0
available = 0
for d in data_arv:
    for hca_id in d:
        short_name = hca_id[:-32]
        obj_uuid = hca_id[-32:]
        try:
            hca = HivCareAdherence.objects.get(id=obj_uuid)
        except HivCareAdherence.DoesNotExist:
            pass
        else:
            arv = Arv.objects.get(short_name=short_name)
            if not arv in hca.arvs.all():
                mis_arv += 1
            else:
                available += 1
print("Missing: ", mis_arv)            
print("Available: ", available)

Missing:  0
Available:  17


In [8]:
# Check if the data has been moved
mis_prev_arv = 0
available_prev_arv = 0
for d in data_prev_arvs:
    for hca_id in d:
        short_name = hca_id[:-32]
        obj_uuid = hca_id[-32:]
        try:
            hca = HivCareAdherence.objects.get(id=obj_uuid)
        except HivCareAdherence.DoesNotExist:
            pass
        else:
            arv = Arv.objects.get(short_name=short_name)
            if not arv in hca.prev_arvs.all():
                mis_prev_arv += 1
            else:
                available_prev_arv += 1
print("Missing prev arv: ", mis_prev_arv)            
print("Available Prev Arv: ", available_prev_arv)

Missing prev arv:  0
Available Prev Arv:  2


In [12]:
still_missing_arvs = []
still_missing_prev_arvs = []
for id in ids:
    try:
        hca = HivCareAdherence.objects.get(id=id)
    except HivCareAdherence.DoesNotExist:
        print("missing", id)
    else:
        if not hca.arvs.all():
            still_missing_arvs.append([hca.subject_visit.subject_identifier, id])
        if not hca.prev_arvs.all():
            still_missing_prev_arvs.append([hca.subject_visit.subject_identifier, id])
print("len arvs: ", len(still_missing_arvs))
print("len prev arvs: ", len(still_missing_prev_arvs))

len arvs:  292
len prev arvs:  311
